In [10]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet/data
## Download
# !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
# !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_velodyne.zip
# !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_calib.zip
# !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip

## Unzip
!unzip data_object_velodyne.zip 'training/*' -d .
!unzip data_object_image_2.zip 'training/*' -d .
!unzip data_object_label_2.zip 'training/*' -d .
!unzip data_object_calib.zip 'training/*' -d .

In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet/data
%pycat crop.py

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet/data


In [ ]:
%%writefile crop.py
import numpy as np
from imageio import imread
from tqdm import tqdm
import glob

CAM = 2

def load_velodyne_points(filename):
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    #points = points[:, :3]  # exclude luminance
    return points

def load_calib(calib_dir):
    # P2 * R0_rect * Tr_velo_to_cam * y
    lines = open(calib_dir).readlines()
    lines = [ line.split()[1:] for line in lines ][:-1]
    #
    P = np.array(lines[CAM]).reshape(3,4)
    #
    Tr_velo_to_cam = np.array(lines[5]).reshape(3,4)
    Tr_velo_to_cam = np.concatenate(  [ Tr_velo_to_cam, np.array([0,0,0,1]).reshape(1,4)  ]  , 0     )
    #
    R_cam_to_rect = np.eye(4)
    R_cam_to_rect[:3,:3] = np.array(lines[4][:9]).reshape(3,3)
    #
    P = P.astype('float32')
    Tr_velo_to_cam = Tr_velo_to_cam.astype('float32')
    R_cam_to_rect = R_cam_to_rect.astype('float32')
    return P, Tr_velo_to_cam, R_cam_to_rect

def prepare_velo_points(pts3d_raw):
    '''Replaces the reflectance value by 1, and tranposes the array, so
        points can be directly multiplied by the camera projection matrix'''
    pts3d = pts3d_raw
    # Reflectance > 0
    indices = pts3d[:, 3] > 0
    pts3d = pts3d[indices ,:]
    pts3d[:,3] = 1
    return pts3d.transpose(), indices

def project_velo_points_in_img(pts3d, T_cam_velo, Rrect, Prect):
    '''Project 3D points into 2D image. Expects pts3d as a 4xN
        numpy array. Returns the 2D projection of the points that
        are in front of the camera only an the corresponding 3D points.'''
    # 3D points in camera reference frame.
    pts3d_cam = Rrect.dot(T_cam_velo.dot(pts3d))
    # Before projecting, keep only points with z>0
    # (points that are in fronto of the camera).
    idx = (pts3d_cam[2,:]>=0)
    pts2d_cam = Prect.dot(pts3d_cam[:,idx])
    return pts3d[:, idx], pts2d_cam/pts2d_cam[2,:], idx


def align_img_and_pc(img_dir, pc_dir, calib_dir):
    
    img = imread(img_dir)
    pts = load_velodyne_points( pc_dir )
    P, Tr_velo_to_cam, R_cam_to_rect = load_calib(calib_dir)

    pts3d, indices = prepare_velo_points(pts)
    pts3d_ori = pts3d.copy()
    reflectances = pts[indices, 3]
    pts3d, pts2d_normed, idx = project_velo_points_in_img( pts3d, Tr_velo_to_cam, R_cam_to_rect, P  )
    #print reflectances.shape, idx.shape
    reflectances = reflectances[idx]
    #print reflectances.shape, pts3d.shape, pts2d_normed.shape
    assert reflectances.shape[0] == pts3d.shape[1] == pts2d_normed.shape[1]

    rows, cols = img.shape[:2]

    points = []
    for i in range(pts2d_normed.shape[1]):
        c = int(np.round(pts2d_normed[0,i]))
        r = int(np.round(pts2d_normed[1,i]))
        if c < cols and r < rows and r > 0 and c > 0:
            color = img[r, c, :]
            point = [ pts3d[0,i], pts3d[1,i], pts3d[2,i], reflectances[i], color[0], color[1], color[2], pts2d_normed[0,i], pts2d_normed[1,i]  ]
            points.append(point)

    points = np.array(points)
    return points

# update the following directories
IMG_ROOT = 'training/image_2/'
PC_ROOT = 'training/velodyne/'
CALIB_ROOT = 'training/calib/'

for frame in tqdm(range(0, 7481)):
    img_dir = IMG_ROOT + '%06d.png' % frame
    pc_dir = PC_ROOT + '%06d.bin' % frame
    calib_dir = CALIB_ROOT + '%06d.txt' % frame

    points = align_img_and_pc(img_dir, pc_dir, calib_dir)
    
    output_name = PC_ROOT + '%06d.bin' % frame
    points[:,:4].astype('float32').tofile(output_name)

Writing crop.py


In [ ]:
!python crop.py

100% 7481/7481 [2:18:23<00:00,  1.11s/it]


In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet/data
%cp -av training validation

In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet/data



/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet/data
number of training files: 59857
number of validation files: 59857


In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet/data
%pycat split.py

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet/data


In [ ]:
%%writefile split.py

import os
import glob

file_num = []
for dir in ['training', 'validation']:
    i = 0
    for _ in glob.iglob(dir + '/**/**', recursive=True):
      i += 1

    print("number of {} files: {}".format(dir, i))
    file_num.append(i)

assert(file_num[0] == file_num[1])

lines_train = [line.rstrip('\n') for line in open('train.txt')]
lines_val = [line.rstrip('\n') for line in open('val.txt')]

for i in lines_train:
  os.remove('training/image_2/'+i+'.png')
  os.remove('training/label_2/'+i+'.txt')
  os.remove('training/velodyne/'+i+'.bin')
  os.remove('training/calib/'+i+'.txt')

for i in lines_val:
  os.remove('validation/image_2/'+i+'.png')
  os.remove('validation/label_2/'+i+'.txt')
  os.remove('validation/velodyne/'+i+'.bin')
  os.remove('validation/calib/'+i+'.txt')

for dir in ['training', 'validation']:
    i = 0
    for _ in glob.iglob(dir + '/image_2/**/**', recursive=True):
      i += 1
    print("number of {} image files: {}".format(dir, i))

    i = 0
    for _ in glob.iglob(dir + '/label_2/**/**', recursive=True):
      i += 1
    print("number of {} lable files: {}".format(dir, i))
    
    i = 0
    for _ in glob.iglob(dir + '/velodyne/**/**', recursive=True):
      i += 1
    print("number of {} velodyne files: {}".format(dir, i))

    i = 0
    for _ in glob.iglob(dir + '/calib/**/**', recursive=True):
      i += 1
    print("number of {} calib files: {}".format(dir, i))

Overwriting split.py


In [ ]:
!python split.py

number of training files: 37585
number of validation files: 37243
number of training image files: 3770
number of training lable files: 3770
number of training velodyne files: 3770
number of training calib files: 3770
number of validation image files: 3713
number of validation lable files: 3713
number of validation velodyne files: 3713
number of validation calib files: 3713


In [9]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet
!zip -r voxelnet_data.zip data

Streaming output truncated to the last 5000 lines.
updating: data/validation/velodyne/003418.bin (deflated 43%)
updating: data/validation/velodyne/003420.bin (deflated 39%)
updating: data/validation/velodyne/003423.bin (deflated 41%)
updating: data/validation/velodyne/003424.bin (deflated 41%)
updating: data/validation/velodyne/003427.bin (deflated 38%)
updating: data/validation/velodyne/003431.bin (deflated 38%)
updating: data/validation/velodyne/003433.bin (deflated 42%)
updating: data/validation/velodyne/003436.bin (deflated 40%)
updating: data/validation/velodyne/003437.bin (deflated 41%)
updating: data/validation/velodyne/003438.bin (deflated 40%)
updating: data/validation/velodyne/003439.bin (deflated 41%)
updating: data/validation/velodyne/003440.bin (deflated 41%)
updating: data/validation/velodyne/003442.bin (deflated 41%)
updating: data/validation/velodyne/003441.bin (deflated 41%)
updating: data/validation/velodyne/003444.bin (deflated 42%)
updating: data/validation/velodyne